In [10]:
from pymongo import MongoClient
from textblob import TextBlob

# emoticons list
neg_emo = ['>:[', ':-(', ':(', ':-c', ':c', ':-<\n', ':?C', ':<', ':-[', ':[', ':{', ':-||', ':@', ":'-(", ":'(", 'D:<', 'D:', 'D8', 'D;', 'D=', 'DX', 'v.v', "D-':", '>:O', ':-O', ':O', '∞o∞', '∞O∞', ':O', 'o_O', 'o_0', 'o.O', '8-0', '>:\\', '>:/', ':-/', ':-.', ':/', ':\\', '=/', '=\\', ':L', '=L', ':S', '>.<', ':-|', ':$', ':-X', ':X', ':-#', ':#', '>:)', '>;)', '>:-)', '}:-)', '}:)', '3:-)', '3:)', ':-&', ':&', '#-)', '%-)', '%)', ':-###..', ':###..', '<:-|', '</3', ':-\\\\', ':\\\\', ':-/', ':/', ')-:', '):', ';-(', ';(']
pos_emo = [':-)', ':)', ':o)', ':]', ':3', ':c)',':>','=]','8)','=)',':}',':^)',':?)',':-D',':D','8-D','8D','x-D','xD','X-D','XD','=-D','=D','=-3','=3','B^D',":-))",'(-:','(:','B-)',';-)',';)',':-P',':P','<3',":'-)",":')",':*',':^*',"('}{' )",'*-)','*)',';-]',';]',';D',';^)',':-,','>:P','X-P','x-p','xp','XP',':-p',':p','=p',':-\xde',':\xde',':-b',':b','O:-)','0:-3','0:3 ','0:-)','0:)','0;^)','o/\\o','^5 ','>_>^ ^<_<','|;-)','|-O']

# totalEmo: makes a list of the pos and neg emoticones
totalEmo = pos_emo + neg_emo
totalEmo2 = set(totalEmo)

# _______________________________________
# Punctuation marks to remove
punc = (['{', '}', '"'])
exclude = set(punc)

# _______________________________________
#connect to Mongodb

client = MongoClient()  
# Connect to a Database:    <<<<<<<<< Modify:ENTER YOUR DB NAME
db = client.tweetsdb_2

# Connect to a Collection:  <<<<<<<<< Modify:ENTER YOUR COLLECTION NAME
tweets = db.modtweet

# Cleans the collection for the modified tweets - extra step
db.pos_tweet.drop()
db.neg_tweet.drop()
db.neu_tweet.drop()

# _______________________________________
# Import Words for Classifier into a list

### Lexicon from first source 
with open('word/pos.txt', 'r') as f:
    positive_words = [line.strip() for line in f]
    positive_words = [x.lower() for x in positive_words]

with open('word/neg.txt', 'r') as f:
    negative_words = [line.strip() for line in f]
    negative_words = [x.lower() for x in negative_words]

### Lexicon from Second source: Bing liu
with open('word/pos1.txt', 'r') as f:
    positive_words1 = [line.strip() for line in f]

with open('word/neg1.txt', 'r') as f:
    negative_words1 = [line.strip() for line in f]

#### Join the two lexicon in one list:
total_positive = positive_words + positive_words1
total_negative = negative_words + negative_words1

# _______________________________________
# Make a list from the Collection for only the tweet :
tw = tweets.find({},{'text':1,'_id':0})
print('Working...')
for x in tw:
    
    tw_string = x['text']
    
    # Remove punctuation:
    # 1. Temporary emoticon variable storage
    emoticonExt = ' '.join(ch for ch in totalEmo2 if ch in tw_string)

    # 2. Remove Punctuation
    json_string = ''.join(ch for ch in tw_string if ch not in exclude)

    # 3. Temporary remove the emoticon
    json_string = json_string +' '+ emoticonExt
    emoticonExt = ""
   
    # _______________________________________
    # First Classifier: Emoticon Classifier

    raw_words = json_string.split(" ")

    positive_score1 = len([word for word in raw_words if word in pos_emo])
    negative_score1 = len([word for word in raw_words if word in neg_emo])

    total_score1 = positive_score1 - negative_score1
    
    if (total_score1 > 0):
        #saving data to Positive Collection
        db.pos_tweet.insert_one({"text":json_string})
               
    elif (total_score1 < 0):
        #saving data to Negative Collection
        db.neg_tweet.insert_one({"text":json_string})
        
    else: # go to next classifier
        
    # _______________________________________
    # Second Classifier: Improved Polarity Classifier

        positive_score2 = len([word for word in raw_words if word in total_positive])
        negative_score2 = len([word for word in raw_words if word in total_negative])

        total_score2 = positive_score2 - negative_score2
        
        if (total_score2 > 0):
            #saving data to Positive Collection
            db.pos_tweet.insert_one({"text":json_string})
            
        elif (total_score2 < 0):
            #saving data to Negative Collection
            db.neg_tweet.insert_one({"text":json_string})
         
        else: # go to next classifier
            
    # _______________________________________
    # Third Classifier: Blob Classifier
                        
            tb = TextBlob(json_string)    
            pol = tb.sentiment.polarity
            if (pol > 0):
                #saving data to Positive Collection
                db.pos_tweet.insert_one({"text":json_string})
               
            elif (pol < 0):
                #saving data to Negative Collection
                db.neg_tweet.insert_one({"text":json_string})
                                
            else: 
                #saving data to Neutral Collection
                db.neu_tweet.insert_one({"text":json_string})
                     
print('Done.') 

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xef in position 3984: invalid continuation byte

In [7]:
with open('word/pos.txt', 'r') as f:
    positive_words = [line.strip() for line in f]
    print('pos1: ', positive_words)
    positive_words = [x.lower() for x in positive_words]
    print('pos2: ', positive_words)

pos1:  ['ABLE', 'ABUNDANCE', 'ABUNDANT', 'ACCLAIMED', 'ACCOMPLISH', 'ACCOMPLISHED', 'ACCOMPLISHES', 'ACCOMPLISHING', 'ACCOMPLISHMENT', 'ACCOMPLISHMENTS', 'ACHIEVE', 'ACHIEVED', 'ACHIEVEMENT', 'ACHIEVEMENTS', 'ACHIEVES', 'ACHIEVING', 'ADEQUATELY', 'ADVANCEMENT', 'ADVANCEMENTS', 'ADVANCES', 'ADVANCING', 'ADVANTAGE', 'ADVANTAGED', 'ADVANTAGEOUS', 'ADVANTAGEOUSLY', 'ADVANTAGES', 'ALLIANCE', 'ALLIANCES', 'ASSURE', 'ASSURED', 'ASSURES', 'ASSURING', 'ATTAIN', 'ATTAINED', 'ATTAINING', 'ATTAINMENT', 'ATTAINMENTS', 'ATTAINS', 'ATTRACTIVE', 'ATTRACTIVENESS', 'BEAUTIFUL', 'BEAUTIFULLY', 'BENEFICIAL', 'BENEFICIALLY', 'BENEFIT', 'BENEFITED', 'BENEFITING', 'BENEFITTED', 'BENEFITTING', 'BEST', 'BETTER', 'BOLSTERED', 'BOLSTERING', 'BOLSTERS', 'BOOM', 'BOOMING', 'BOOST', 'BOOSTED', 'BREAKTHROUGH', 'BREAKTHROUGHS', 'BRILLIANT', 'CHARITABLE', 'COLLABORATE', 'COLLABORATED', 'COLLABORATES', 'COLLABORATING', 'COLLABORATION', 'COLLABORATIONS', 'COLLABORATIVE', 'COLLABORATOR', 'COLLABORATORS', 'COMPLIMENT', 'C